## TODO
- Create column "month qty to expire"

In [2]:
from Utils.DataLoader import DataLoader
from Utils.DuckDb import DuckDb
from datetime import datetime
import pandas as pd
from typing import List

In [3]:
DUCK_DB_UTILS = DuckDb()

In [11]:
MININUM_USERS_TO_CONSIDER = 50_000
BATCH_SIZE_TO_GET_DATA_FROM_DATABASE = 20_000

In [12]:
def get_user_list(limit: int = 9999999999, offset: int = 0) -> List[str]:
    conn = DUCK_DB_UTILS.get_connection()
    query = '''
        SELECT DISTINCT(msno)
        FROM main.user_logs ul
        ORDER BY msno
        LIMIT ?
        OFFSET ?
    '''
    query_results = conn.execute(query, [limit, offset]).fetchall()
    result = list(
        map(
            lambda qr: qr[0], query_results
        )
    )
    return result

In [13]:
def get_dataset_by_users(msnos: List[str]) -> pd.DataFrame:
    query = '''
        SELECT
            ----------------------
            -- Calculated fields --
            ----------------------
            50 + (0.0051 * num_unq) + (0.0001 * ul.total_secs) AS cost,
            --	t.actual_amount_paid - cost AS net_profit,
            
            ----------------------
            -- User Logs fields --
            ----------------------
            ul.msno,
            ul.safra,
            ul.num_25,
            ul.num_50,
            ul.num_75,
            ul.num_985,
            ul.num_100,
            ul.num_unq,
            ul.total_secs,
            ul.total_hours,
            
            -------------------------
            -- Transactions fields --
            -------------------------
            t.msno,
            t.payment_method_id,
            t.payment_plan_days,
            t.plan_list_price,
            t.actual_amount_paid,
            t.is_auto_renew,
            t.is_cancel,
            t.safra,
            t.transaction_date_year,
            t.transaction_date_month,
            t.transaction_date_day,
            t.transaction_date_day_of_week,
            t.transaction_date_day_of_year,
            t.membership_expire_date_year,
            t.membership_expire_date_month,
            t.membership_expire_date_day,
            t.membership_expire_date_day_of_week,
            t.membership_expire_date_day_of_year,
            t.discount,
            t.price_per_month,
            
            ---------------------
            -- Members columns --
            ---------------------
            m.msno,
            m.safra,
            m.city,
            m.registered_via,
            m.is_active,
            m.registration_init_time_year,
            m.registration_init_time_month,
            m.registration_init_time_day,
            m.registration_init_time_day_of_week,
            m.registration_init_time_day_of_year
        FROM
            main.user_logs ul
        INNER JOIN
            main.transactions t ON
            t.msno == ul.msno
            AND t.safra == ul.safra
        INNER JOIN
            main.members m ON
            m.msno = ul.msno AND m.safra = ul.safra
        WHERE
            ul.msno IN ?
            AND
            ul.safra < 201701
        ORDER BY
            ul.msno,
            ul.safra
    '''

    conn = DUCK_DB_UTILS.get_connection()
    query_results = conn.execute(query, (msnos,)).fetch_df()
    return query_results

In [14]:
def upload_treated_dataframe_to_duck_db(df: pd.DataFrame):
    conn = DUCK_DB_UTILS.get_connection()
    datetime_string_identifier = datetime.now().strftime('%Y_%m_%d_%Hh%mm')
    table_name = 'treated_churn_dataset_' + datetime_string_identifier
    temp_table = 'temp_' + datetime_string_identifier

    conn.register(temp_table, df)
    conn.execute(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM {temp_table}")

    print(f'Inseridos registros na tabela {table_name}')


In [15]:
users_msno = get_user_list(limit=10, offset=0)

df = get_dataset_by_users(users_msno)

In [16]:
def get_dataset() -> pd.DataFrame:
    all_dfs: List[pd.DataFrame] = []

    count = 0
    while count < MININUM_USERS_TO_CONSIDER:
        print(f'Processando count: {count}')

        users_msno = get_user_list(limit=BATCH_SIZE_TO_GET_DATA_FROM_DATABASE, offset=count)
        count += BATCH_SIZE_TO_GET_DATA_FROM_DATABASE

        all_dfs.append(
            get_dataset_by_users(users_msno)
        )

    print(f'Qtd. de dataframes: {len(all_dfs)}')

    all_dfs = list(
        filter(
            lambda df: df.__len__() > 0, all_dfs
        )
    )

    print(f'Qtd. de dataframes pós remoção dos vazios: {len(all_dfs)}')

    result = pd.concat(all_dfs)
    return result

In [17]:
full_dataframe = get_dataset()

Processando count: 0


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Processando count: 20000


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Processando count: 40000


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Qtd. de dataframes: 3
Qtd. de dataframes pós remoção dos vazios: 3


In [18]:
users_msno = list(full_dataframe['msno'].unique())

In [19]:
def get_next_safras(safra: int, month_qty: int) -> int:
    str_safra = str(safra)
    year, month = int(str_safra[:4]), int(str_safra[4:])

    month -= 1

    month += month_qty

    year += month // 12
    month = month % 12

    month += 1

    formatted_month = f'0{month}' if month < 10 else (month)
    return int(f'{year}{formatted_month}')


In [20]:
def calc_churn(df: pd.DataFrame, users_msno: List[str]) -> pd.DataFrame:
    df_by_users = {}

    print(f'Separando DataFrames por usuários')
    for index, usr in enumerate(users_msno):
        if index % 1000 == 0:
            print(f'-> {index} / {len(users_msno)}')
            
        df_by_users[usr] = df[df['msno'] == usr]

    def __calc_row_churn(user_df, row):
        months_to_consider_churn = 3

        is_churn = False
        should_delete_row = False
        # print(f'Safra atual: {row["safra"]}')
        for m in range(1, months_to_consider_churn + 1):
            next_safra = get_next_safras(row['safra'], m)
            # print(f'Recuperando informações da safra {next_safra}')

            # print(user_df['safra'])
            next_safra_row = user_df[user_df['safra'] == next_safra].reset_index()

            if len(next_safra_row) == 0:
                # print('Safra não encontrada, pulando')
                should_delete_row = True
                break

            if next_safra_row['is_cancel'][0] == True:
                # print('Safra encontrada com is_cancel, marcando como churn!')
                is_churn = True
                break

        row['is_churn'] = is_churn
        row['should_delete_row'] = should_delete_row
        return row


    rows = []
    users_qty = len(df_by_users.values())
    count = 0
    for msno, user_df in df_by_users.items():
        # print(f'Processando usuário {msno}')

        count += 1
        print(f'Processando usuário {count}/{users_qty} ({msno})')

        for _, user_row in user_df.iterrows():
            user_row = __calc_row_churn(user_df, user_row)
            rows.append(user_row)
            # print(user_row)

    result = pd.DataFrame(rows)
    return result

In [21]:
treated_df = calc_churn(
    full_dataframe,
    users_msno
)

Separando DataFrames por usuários
-> 0 / 15189
-> 1000 / 15189
-> 2000 / 15189
-> 3000 / 15189
-> 4000 / 15189
-> 5000 / 15189
-> 6000 / 15189
-> 7000 / 15189
-> 8000 / 15189
-> 9000 / 15189
-> 10000 / 15189
-> 11000 / 15189
-> 12000 / 15189
-> 13000 / 15189
-> 14000 / 15189
-> 15000 / 15189
Processando usuário 1/15189 (+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=)
Processando usuário 2/15189 (+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=)
Processando usuário 3/15189 (+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=)
Processando usuário 4/15189 (+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=)
Processando usuário 5/15189 (++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=)
Processando usuário 6/15189 (++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=)
Processando usuário 7/15189 (++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=)
Processando usuário 8/15189 (++/gTmVgKUbNFmsTiriZdWV1uZIrLXCUiEWN0fEU6BM=)
Processando usuário 9/15189 (++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=)
Processando usuário 10/15189 (++

In [22]:
debug_df = treated_df[treated_df['is_churn'] == True]
debug_df[['msno', 'safra', 'is_cancel', 'is_churn']]

,msno,safra,is_cancel,is_churn
100,++2axpngZEynlxNr1+AkwgHHfaEZ/EeOj6Q284RiAkw=,201601,False,True
173,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,201601,False,True
174,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,201602,False,True
244,++8dXbkKMJ0rXwUc/m19lTVokEl3c9EfRKWmV6qP9jg=,201605,False,True
245,++8dXbkKMJ0rXwUc/m19lTVokEl3c9EfRKWmV6qP9jg=,201606,False,True
...,...,...,...,...
31294,+ix2IOBJSGRHtGyltLD9DprOAd8Y60t4ns8YBz5W/EU=,201602,False,True
31295,+ix2IOBJSGRHtGyltLD9DprOAd8Y60t4ns8YBz5W/EU=,201603,False,True
31356,+iz30Nm1c+4V8DAVMWAbH2zeMN4Jto8FWJD2BGnjqw8=,201609,False,True
31357,+iz30Nm1c+4V8DAVMWAbH2zeMN4Jto8FWJD2BGnjqw8=,201610,False,True


In [23]:
debug_df = treated_df[treated_df['is_churn'] == True]
debug_df.describe()

,cost,safra,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,total_hours,...,membership_expire_date_day_of_week,membership_expire_date_day_of_year,discount,price_per_month,safra_2,registration_init_time_year,registration_init_time_month,registration_init_time_day,registration_init_time_day_of_week,registration_init_time_day_of_year
count,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,...,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000
mean,63.512183,201606.047459,101.706833,26.014137,16.273645,17.616964,421.128913,437.458432,112811.450353,31.336514,...,2.971727,200.148435,6.827331,137.730575,201606.047459,2012.623023,6.287445,15.277348,3.045776,175.486032
std,13.602888,3.005176,145.602611,33.930248,19.561029,22.552625,449.849752,434.005975,115824.773590,32.173548,...,1.992040,94.197878,31.146717,34.471061,3.005176,2.769713,3.517657,8.790442,1.931815,107.414560
min,50.010200,201601.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,51.000000,0.014167,...,0.000000,1.000000,0.000000,0.000000,201601.000000,2004.000000,1.000000,1.000000,0.000000,1.000000
25%,53.725100,201603.000000,19.000000,5.000000,4.000000,4.000000,106.000000,126.000000,29822.000000,8.283889,...,1.000000,118.500000,0.000000,149.000000,201603.000000,2011.000000,3.000000,8.000000,1.000000,80.000000
50%,59.303000,201607.000000,53.000000,15.000000,10.000000,10.000000,275.000000,309.000000,76460.000000,21.238889,...,3.000000,213.000000,0.000000,149.000000,201607.000000,2013.000000,6.000000,15.000000,3.000000,172.000000
75%,68.491700,201609.000000,126.000000,33.000000,22.000000,23.000000,571.500000,604.000000,153035.000000,42.509722,...,5.000000,279.000000,0.000000,149.000000,201609.000000,2015.000000,9.000000,22.000000,5.000000,270.000000
max,131.328200,201611.000000,2036.000000,356.000000,208.000000,249.000000,3097.000000,3565.000000,654418.000000,181.782778,...,6.000000,366.000000,149.000000,180.000000,201611.000000,2016.000000,12.000000,31.000000,6.000000,365.000000


In [24]:
treated_df[treated_df['is_churn'] == False].describe()

,cost,safra,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,total_hours,...,membership_expire_date_day_of_week,membership_expire_date_day_of_year,discount,price_per_month,safra_2,registration_init_time_year,registration_init_time_month,registration_init_time_day,registration_init_time_day_of_week,registration_init_time_day_of_year
count,89676.000000,89676.000000,89676.000000,89676.000000,89676.000000,89676.000000,89676.000000,89676.000000,89676.000000,89676.000000,...,89676.000000,89676.000000,89676.000000,89676.000000,89676.000000,89676.000000,89676.000000,89676.000000,89676.000000,89676.000000
mean,63.590588,201606.890617,107.258977,25.983730,16.155225,17.685791,422.311521,443.585742,113283.002743,31.467501,...,2.954581,189.745116,0.578271,131.283935,201606.890617,2012.835385,6.517586,15.565034,3.111981,182.859093
std,14.263325,3.470579,163.703229,34.764884,20.313661,27.978172,469.808925,456.690144,121424.521694,33.729034,...,1.993175,109.654888,9.228660,32.944201,3.470579,2.939240,3.586704,8.734309,1.993196,109.573134
min,50.005200,201601.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000278,...,0.000000,1.000000,0.000000,0.000000,201601.000000,2004.000000,1.000000,1.000000,0.000000,1.000000
25%,53.154675,201604.000000,18.000000,5.000000,3.000000,3.000000,88.000000,108.000000,25373.750000,7.048264,...,1.000000,91.000000,0.000000,99.000000,201604.000000,2011.000000,3.000000,8.000000,1.000000,80.000000
50%,58.900600,201607.000000,53.000000,14.000000,9.000000,10.000000,262.000000,298.000000,72622.500000,20.172917,...,3.000000,197.000000,0.000000,149.000000,201607.000000,2013.000000,7.000000,16.000000,3.000000,188.000000
75%,69.183725,201610.000000,130.000000,33.000000,21.000000,22.000000,587.000000,630.000000,158572.750000,44.047986,...,5.000000,287.000000,0.000000,149.000000,201610.000000,2015.000000,10.000000,23.000000,5.000000,281.000000
max,132.368800,201612.000000,3603.000000,392.000000,214.000000,1382.000000,4415.000000,4136.000000,655185.000000,181.995833,...,6.000000,366.000000,149.000000,180.000000,201612.000000,2016.000000,12.000000,31.000000,6.000000,366.000000


In [25]:
msno = '+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY='
treated_df[treated_df['msno'] == msno][['safra', 'is_cancel', 'is_churn', 'is_active', 'should_delete_row']]

,safra,is_cancel,is_churn,is_active,should_delete_row
0,201609,False,False,True,True


In [26]:
for usr in users_msno:
    found = df[df['msno'] == usr]
    if len(found) > 0:
        print(usr)
        print(found['is_cancel'])


+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=
0    False
Name: is_cancel, dtype: bool
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=
1    False
2    False
Name: is_cancel, dtype: bool
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: is_cancel, dtype: bool


In [27]:
upload_treated_dataframe_to_duck_db(treated_df)

Inseridos registros na tabela treated_churn_dataset_2025_03_11_08h03m
